In [1]:
filename= "https://github.com/guedalia/binder/raw/master/talmud_topics_clean.csv"

In [2]:
import pandas as pd

In [3]:
import plotly.graph_objects as go

In [4]:
from ipywidgets import interact

In [5]:
from collections import Counter

In [6]:
topics = pd.read_csv(filename)

In [7]:
def split_book_to_chunks(book_daf_min, book_daf_max, step, my_topics, num_most_common): 
  l=[]

  for x in range(book_daf_min, book_daf_max, step):
    slugs = my_topics[(my_topics["daf_int"] >= x)&(my_topics["daf_int"] < x+step)]["Slug"]
    counted_slugs = Counter(slugs)
    if x+step > book_daf_max:
      new_step =  book_daf_max
    else:
      new_step =  x + step
    
    for top_slug in counted_slugs.most_common(num_most_common):
      d={
        'chunk': (x, new_step),
        'topic': top_slug[0],
        'count': top_slug[1]
      }
      l.append(d)
      
  return(l)
    # print(x, new_step, counted_slugs.most_common(5))

In [10]:
@interact(chosen_book = set(topics["book"]),  step = (1, 200, 1), num_most_common=(1,10,1))
def topic_by_daf(chosen_book= "Sukkah", step = 10, num_most_common =1):
  my_topics = topics[topics["book"]==chosen_book]
  
  book_daf_min = min(my_topics["daf_int"])
  book_daf_max = max(my_topics["daf_int"])

  l = split_book_to_chunks(book_daf_min, book_daf_max, step, my_topics, num_most_common)

  source = []
  target = []
  count = []

  for x in l:
    source.append(x['chunk'])
    target.append(x['topic'])
    count.append(x['count'])

  color_lookup = {}
  for i,x in enumerate(set(target)):
    color_lookup[x] = i

  color_list = []
  for x in target:
    color_list.append(color_lookup[x])
  

  fig = go.Figure(
    go.Parcats(
        dimensions=[
             {'label': 'Dapim',
             'values': source},
             {'label': 'Topics',
             'values': target},
        ],
        counts=count,
        line={'color': color_list}
      )
  )
  fig.update_layout(
      title = chosen_book
  )

  fig.show()

interactive(children=(Dropdown(description='chosen_book', index=15, options=('Bava Metzia', 'Ketubot', 'Berakh…